<a href="https://colab.research.google.com/github/jekim526/IE-541-Project/blob/main/BasicGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize
Load problem data, install packages

In [1]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
!pip install deap
!git clone https://github.com/jekim526/IE-541-Project.git
!git --git-dir=/content/IE-541-Project/.git pull
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
sys.path.append('/content/IE-541-Project')
import knapsack_EA_functions as ea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 139 kB 5.3 MB/s 
Cloning into 'IE-541-Project'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 263 (delta 78), reused 120 (delta 70), pack-reused 126
Receiving objects: 100% (263/263), 2.86 MiB | 6.35 MiB/s, done.
Resolving deltas: 100% (153/153), done.
Already up to date.


In [2]:
url = 'https://raw.githubusercontent.com/jekim526/IE-541-Project/main/data/Z_r_100_25_1.csv' # Get the "Raw" link
df1_o = pd.read_csv(url)
df1 = pd.DataFrame(df1_o).to_numpy() # DataFrame to nummpy array
n_size = len(df1) - 1

# Array of item value
item_value = [0] * n_size
for x in range(0,n_size):
  item_value[x] = df1[1,x+1]

# Array of item weight
item_weight = [0] * n_size
for x in range(0,n_size):
  item_weight[x] = df1[0,x+1]

# n x n array of joint profit
joint_profit = np.zeros((n_size,n_size))
for x in range(0, n_size-1):
  for y in range(0, n_size):
    joint_profit[x,y] = df1[x+2,y+1]

item_weight = np.array(item_weight)
item_value = np.array(item_value)

## Prepare the GA input and solve the instance via GA

In [3]:
# -------  NOTES: ------------------------------------------------------------------------------
# instance_setings should be a tuple contains:
# {item_value, item_weight, joint_profit, capacities}
#                 capacities = {capacity1, capacity2, ...}
#
# evolution_general_parameters should be a tuple contains:
# {popsize, swap_prob, mute_prob, punish_factor}, in which:
#    swap_prob is independent probability for swap at each point in uniform crossover.
#    mute_prob is independent probability for each attribute to be flipped in flip-bit mutation.
#
# evolution_specify_parameters should be a tuple contains: {CXPB, MUTPB},In which:
#    CXPB is the probability with which two individuals
#          are crossed
#    MUTPB is the probability for mutating an individual
#    MAX_GEN is the maximum generation threshold
#    STOP_GEN is the threshold of no progress generations

''' the maximize covalue_case '''
max_c = 0; max_c = max(max_c,max(np.sum(joint_profit,axis = 1))); max_c = max(max_c,max(np.sum(joint_profit,axis = 0)))
max_i = max(item_value)

''' prepare this solver inputs: '''
#initialize evolution parameter setting
evolution_general_parameters = (100, 0.1, 0.1, max_i + max_c)
evolution_specify_parameters = (0.5, 0.25, 1000, 100)
num_of_knapsack = 10
capacity = (sum(item_weight)/num_of_knapsack)*0.8 # 80% of the sum of all item weights divided by the number of knapsacks
capacities = (capacity,)*num_of_knapsack
#initialize instance setting
instance_settings = (item_value, item_weight, joint_profit, capacities)

In [4]:
''' solve via GA in one line! '''
objf_type = 1 
#objf_type: 
# 1: the first objective function (maximum the total profit)
# 2: maximum the negative of total_weight
# 3: maximum the profit of the package with minmum of profit

best_ind,pop,num_gen = ea.perform_GA_base(objf_type, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=True)
best_ind.fitness.values

Start of evolution
  Evaluated 100 individuals
-- Generation 1 --
  Evaluated 64 individuals
  Min -587046.1999999998
  Max -12842.679999999993
  Avg -252471.8019999997
  Std 134344.39057701948
-- Generation 2 --
  Evaluated 63 individuals
  Min -596330.5199999999
  Max -31436.839999999997
  Avg -213236.8859999997
  Std 108270.88753014806
-- Generation 3 --
  Evaluated 52 individuals
  Min -465494.36
  Max -31436.839999999997
  Avg -169508.89679999973
  Std 89356.33226151677
-- Generation 4 --
  Evaluated 62 individuals
  Min -464804.19999999995
  Max 2651.0
  Avg -162753.62759999975
  Std 88538.02629214792
-- Generation 5 --
  Evaluated 67 individuals
  Min -458843.19999999995
  Max 2651.0
  Avg -159343.6119999998
  Std 86870.76339732339
-- Generation 6 --
  Evaluated 60 individuals
  Min -510390.19999999995
  Max 4128.0
  Avg -129919.32159999979
  Std 86395.02932905903
-- Generation 7 --
  Evaluated 69 individuals
  Min -319491.67999999993
  Max 3222.0
  Avg -114038.79199999983
  Std

(9485.0,)

In [ ]:
evolution_general_parameters = (100, 0.1, 0.1, 200)
evolution_specify_parameters = (0.5, 0.25, 1000, 100)
best_ind,pop,num_gen = ea.perform_GA_base(1, num_of_knapsack, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)
best_ind.fitness.values

In [5]:
# def exam_feasibility(individual, instance_settings):
#   result = []
#   item_weight = instance_settings[1]
#   capacities = instance_settings[3]
#   for k in range(individual.shape[1]):
#     kth_knapsack = individual[:,k]
#     weight = np.dot(kth_knapsack, item_weight)
#     if weight > capacities[k]:
#       result.append(False) ## exceed the weight limitation, infeasible
#     else:
#       result.append(True) ## feasible
#   return result
#   # return True means feasible, 
ea.exam_feasibility(best_ind, instance_settings)    

[True, True, True, True, True, True, True, True, True, True]

Since above naive GA amazingly always produce feasibile solution, I want to verify my implementation of `exam_feasibility`. The exam below shows my implementation of `exam_feasibility` is correct.

In [ ]:
def exam_feasibility(individual, item_weight, capacities):
  result = []
  for k in range(individual.shape[1]):
    kth_knapsack = individual[:,k]
    weight = np.dot(kth_knapsack, item_weight)
    if weight > capacities[k]:
      result.append(False) ## exceed the weight limitation, infeasible
    else:
      result.append(True) ## feasible
  return result
  # return True means feasible, 

ind = np.array([[0,0,0],[0,1,0],[1,0,1]]).T # Note: in individual, every column is a decision vector of 
weight = np.array([20,30,40])
caps = (30,30,30)
exam_feasibility(ind,weight,caps)

In [ ]:
#Output the best individual as .csv file
import pandas as pd
df_best_ind = pd.DataFrame(best_ind.T)
df_best_ind.to_csv("/content/best_ind.csv")